0623

#네이버 영화 TOP 100 크롤링

네이버 영화 TOP 100  크롤링 순서<br>
1) 네이버 영화 TOP 100 페이지 별 링크 수집<br>
2) 수집된 영화 링크에 접속하여 영화 정보(제목, 평점, 장르, 줄거리) 크롤링<br>
3) 데이터 분석에 필요한 CSV 파일로 저장

라이브러리 설치

In [ ]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

라이브러리 임포트

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

분석할 데이터 모으기(크롤링)

네이버 영화 TOP 100 페이지별 링크 수집

https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=(1,5) 실시간
https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page=(1,5) 일간
https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page=(1,5) 주간
https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=(1,5) 월간


In [ ]:


def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo -1, lastPageNo) :
    # 실수간 TOP 100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page='+ str(pageNo + 1)

    req = requests.get(url) #requests.get(a) -> a의 정보를 받아올 수 있게 함 ==> 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]')
    # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
    # select는 무언갈 찾는 함수

    for movielink in movielinks :
      link = str(movielink.get('href'))
      links.append('http://series.naver.com' + link)
      # 풀링크를 만들 수 있게 됨(접속할 수 있는 전체 링크 형태로 변환)
      
  return links




#네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links):
  title_infos = []   # 제목
  content_infos = [] # 줄거리
  genre_infos = []   # 장르
  score_infos = []   # 평점
  date_infos = []    # 개봉일

  url2 = 'https://www.naver.com'

  driver = webdriver.Chrome('chromedriver', options=options) # 가상의 driver를 만듦
  driver.get(url2)
  time.sleep(3.0) # 3초 sleep : 시간을 벌어주는 역할

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't') # 컨트롤 t는 새 창여는 단축키
  # body는 화면이라고 생각하면 됨
  
  for link in links:
    print(link + ' 수집 중......')
    driver.switch_to.window(driver.window_handles[-1]) # 새창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새창 활성화
    time.sleep(3.0)

    html_source = driver.page_source # req. text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년관람불가 영화 크롤링 할 경우 -> 인증 필요 ==> 수집 제외
    flag = html_soup.text[0:10]

    newflag = ''.join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em ')
      score = float(score.text)
      score = int(score) # 7.7이 7로 수집됨
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르', '')
      genre = genre.replace('\n', '')
      genre = genre.replace('\t', '')
      genre_infos.append(genre)

      # 제목, 줄거리

      try:
        movieinfoUrl= driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except:
        movieinfoUrl= driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieinfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find('meta', {'property' : 'og:title'}).get('content')
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append('줄거리 오류')
      
      else:
        for content in contents_texts :
          # 줄거리 데이터 클렌징 작업
          temp = content.text
          temp = temp.replace('\r', '')   # 줄바꿈 제거
          temp = temp.replace('\xa0', '') # 공백 제거
          content_infos.append(temp)
      
    elif newflag == '' :
        print('청불 영화로 데이터 수집하지 않습니다.')

  print('수집 완료합니다......')
    # 토탈 정보 찍기
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {'제목' : title_infos, '평점':score_infos, '장르':genre_infos, '줄거리':content_infos}

    # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

    # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates('줄거리', keep='first') # 줄거리 비교해서, 중복된 영화를 삭제(첫번째 수집 영화만 남김)

  return movie_df2



# CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num) :
  try : 
    movie_df.to_csv(('movie_my_data_' + str(num) + '.csv'),
                    sep =',', na_rep ='NaN', encoding='euc-kr')
  
  except :
    print('Error')

# 크롤링 실행

In [ ]:
#1) 크롤링 할 링크 생성

links = getPageLinksWantRange(1,5)

#2) 영화 상세 정보 수집

movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv 파일 저장

dftoCsv(movie_df2, len(movie_df2))

#크롤링한 TOP 100 영화 데이터 랭킹 시각화

라이브러리 임포트

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

데이터 준비하기

#실시간 TOP 100 시각화

In [ ]:
df = pd.read_csv('/content/movie_my_data_66_실시간.csv', encoding='euc-kr')
df.info()

In [ ]:
df.head()

In [ ]:
fig = px.scatter(df, x= '제목', y='평점', color='장르')
fig.show()
# 어떤 장르가 상위권에 있는지 전반적으로 보여줌

In [ ]:
df_score = df[df.평점 == 9.0]
df_score.tail()

# 9점 평점의 영화중 가장 큰 비율을 차지하고 있는 장르는?

In [ ]:
fig = px.pie(df_score, values='평점', names='장르')
fig.show()

#일간 TOP 100 시각화

In [ ]:
df2 = pd.read_csv('/content/movie_my_data_61_일간.csv', encoding='euc-kr')
df2.head()

In [ ]:
df2_score = df2[df2.평점 == 9.0]
df2_score.tail()

In [ ]:
fig = px.pie(df2_score, values='평점', names='장르')
fig.show()

In [ ]:
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()
# 영화가 평점 몇 번 받았는지 확인할 때 용이/ 평점 표현할 때가 용이

In [ ]:
fig = px.bar(df2, x='제목', y='평점')
fig.show()

In [ ]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()
# 시기별 선호 장르 확인에도 응용할 수 있음

#주간 TOP 100 시각화

In [ ]:
df3 = pd.read_csv('/content/movie_my_data_63_주간.csv', encoding='euc-kr')
df3.head()

In [43]:
df3_score = df3[df3.평점 == 0.0]
df3_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
23,29,잭 스나이더의 저스티스 리그,0,액션,슈퍼맨이 죽고 지구에 어둠의 그림자가 드리운다. ‘마더박스’를 차지하기 위해 빌런 ...


In [ ]:
fig = px.bar(df3, x='제목', y='평점', color='장르')
fig.show()

In [ ]:
fig = px.pie(df3, values='평점', names='장르')
fig.show()
# 장르별로 평점을 매기기

In [ ]:
fig = px.scatter(df3, x='제목', y='평점', color='장르')
fig.show()

#월간 TOP 100 시각화

In [ ]:
df4 = pd.read_csv('/content/movie_my_data_64_월간.csv', encoding='euc-kr')
df4.head()

In [ ]:
df4_score = df4[df4.평점 >= 9.0]
df4_score.tail()

In [ ]:
fig = px.pie(df4_score, values='평점', names='장르')
fig.show()

In [ ]:
fig = px.bar(df4, x='제목', y='평점', color='장르')
fig.show()

In [ ]:
fig = px.bar(df4, x='제목', y='평점')
fig.show()

In [51]:
fig = px.pie(df4, values='평점', names='장르')
fig.show()

In [57]:
fig = px.pie(df4, names='장르')
fig.show()
# 월간 TOP100을 장르별로 그룹지어서 파이 차트를 그림

In [59]:
fig = px.pie(df4, values='평점', names='장르')
fig.show()
# 월간 TOP100을 장르별로 그룹짓고 평점을 더해 파이 차트를 그림

In [ ]:
fig = px.scatter(df4, x='제목', y='평점', color='장르')
fig.show()